In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as mse
import importlib
import lightgbm as lgb
#import airbnb_test_func_ver2 as tf
#importlib.reload(tf)

%matplotlib inline

In [4]:
df = pd.read_csv("train_meanprice.csv")

#df = tf.data_pre(df)

#model = tf.model_lgb(df)

In [5]:
df_train, df_val = train_test_split(df, test_size=0.2)
    
col = "n"
train_y = df_train[col]
train_x = df_train.drop(col, axis=1)

val_y = df_val[col]
val_x = df_val.drop(col, axis=1)

trains = lgb.Dataset(train_x, train_y)
valids = lgb.Dataset(val_x, val_y)

params = {
    "task": "train", 
    "objective": "regression",
    "boosting_type": "gbdt", 
     "metrics": {"rmse"}, 
    "learning_rate": 0.09789358290638106, 
    "num_leaves": 15, 
    "tree_learner": "feature",
    "lambda_l1": 129.61046396066124, 
    "lambda_l2": 92.07795163948921, 
    "seed": 17, 
    "max_depth": 6
    }

categorical_list = [
                    "Month",
                    "shopID",
                    "Cat",
                    "itemID",
                    ]

model = lgb.train(params, 
                    trains, 
                    valid_sets=valids, 
                    categorical_feature=categorical_list, 
                    num_boost_round=1000, 
                    early_stopping_rounds=50, 
                    )

c:\Users\NDY00\miniforge3\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
c:\Users\NDY00\miniforge3\lib\site-packages\lightgbm\basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['Cat', 'Month', 'itemID', 'shopID']
  _log_warning('categorical_feature in Dataset is overridden.\n'
c:\Users\NDY00\miniforge3\lib\site-packages\lightgbm\basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
c:\Users\NDY00\miniforge3\lib\site-packages\lightgbm\basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is over

[LightGBM] [Warning] Met categorical feature which contains sparse values. Consider renumbering to consecutive integers started from zero
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000362 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 456
[LightGBM] [Info] Number of data points in the train set: 53856, number of used features: 5
[LightGBM] [Info] Start training from score 0.747976
[1]	valid_0's rmse: 1.74318
Training until validation scores don't improve for 50 rounds
[2]	valid_0's rmse: 1.71697
[3]	valid_0's rmse: 1.6941
[4]	valid_0's rmse: 1.67454
[5]	valid_0's rmse: 1.65637
[6]	valid_0's rmse: 1.64132
[7]	valid_0's rmse: 1.6276
[8]	valid_0's rmse: 1.61531
[9]	valid_0's rmse: 1.60493
[10]	valid_0's rmse: 1.59525
[11]	valid_0's rmse: 1.58696
[12]	valid_0's rmse: 1.57794
[13]	valid_0's rmse: 1.5701
[14]	valid_0's rmse: 1.55992


In [6]:
'''
vals = model.predict(val_x)
rmse = np.sqrt(mse(val_y, vals))
rmse
#モデル構築で帰ってきたRMSE値と一致してるか
'''

'\nvals = model.predict(val_x)\nrmse = np.sqrt(mse(val_y, vals))\nrmse\n#モデル構築で帰ってきたRMSE値と一致してるか\n'

In [8]:
#いよいよテストデータを予測してsubmit.csvに書き出し
df_test = pd.read_csv("test_meanprice.csv")
#df_test = tf.data_pre(df_test)

predict = model.predict(df_test)
df_test["n"] = predict
#df_test["y"].to_csv("submit.csv", header = None) #応募用ファイルはヘッダーなし

In [9]:
#lightgbmのfeature_importance関数を使って特徴量重要度を見てみる
pd.DataFrame(model.feature_importance(), index=val_x.columns, columns=["importance"]).sort_values("importance", ascending=False)

,importance
MeanPrice,2867
itemID,1453
shopID,839
Month,375
Cat,37


In [13]:
df_test["n"].to_csv("submit8.csv", index=True, header=False)